In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
import joblib

In [2]:
data_property_cell = pd.read_csv("../data/qmof_property_cell.csv", index_col=1) # index_col=1 id "filename"
data_property_cell = data_property_cell.drop("Unnamed: 0", axis=1) # drop numbers of rows
property_cell_cols = ["pld", "lcd", "a", "b", "c", "alpha", "betta", "gamma", "volume", "spacegroupNumber"]
data_property_cell = data_property_cell.loc[:,property_cell_cols].rename({"betta": "beta"}, axis=1)

In [3]:
data_linker_mordred = pd.read_csv("../data/mordred_descriptors.csv", index_col=0)

In [4]:
p = len(data_linker_mordred.values[np.isnan(data_linker_mordred)]) / len(data_linker_mordred) / len(data_linker_mordred.iloc[0])
print(f"Mordred data has {p*100:.2n} % of nan")

Mordred data has 25 % of nan


In [5]:
data_linker_mordred.shape

(7109, 1826)

Load node data

In [6]:
data_node = pd.read_csv("../data/node_descriptors.csv", index_col=0)
data_node = data_node.loc[data_node["n_types_metals"] == 1.]
data_node = data_node.loc[:, ['n_metals', 'Atomic_Number', 'Atomic_Weight',
                              'Atomic Radius', 'Mulliken EN', 'polarizability(A^3)', 'electron affinity(kJ/mol)']]


In [7]:
data_qmof = pd.concat([data_property_cell, data_linker_mordred, data_node], axis=1)

In [8]:
data_qmof.shape

(7463, 1843)

In [9]:
data_qmof = data_qmof.loc[sorted(list({*list(data_linker_mordred.index)} & {*list(data_node.index)}))]

In [10]:
data_qmof.shape

(6797, 1843)

In [11]:
data_qmof.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6797 entries, ABAVIJ_FSR to tobacco_srsb_sym_3_on_5_sym_3_mc_0_L_2
Columns: 1843 entries, pld to electron affinity(kJ/mol)
dtypes: float64(1842), int64(1)
memory usage: 95.6+ MB


In [48]:
train, test = train_test_split(data_qmof, test_size=0.2, random_state=0)

In [49]:
train.shape

(5437, 1843)

In [50]:
len(train["beta"].dropna())

5437

In [51]:
train["pld"].std()

4.2136908756426505

In [52]:
for col in train.columns:
    print(f"{col} has {((5437 - len(train[col].dropna()))/5437)*100:.0f} % of nan and {train[col].std():.4f} std")

pld has 0 % of nan and 4.2137 std
lcd has 0 % of nan and 5.0673 std
a has 0 % of nan and 4.7709 std
b has 0 % of nan and 4.7470 std
c has 0 % of nan and 5.9542 std
alpha has 0 % of nan and 14.2085 std
beta has 0 % of nan and 12.2731 std
gamma has 0 % of nan and 14.4234 std
volume has 0 % of nan and 3204.7636 std
spacegroupNumber has 0 % of nan and 43.3511 std
ABC has 0 % of nan and 6.8132 std
ABCGG has 0 % of nan and 4.9800 std
nAcid has 0 % of nan and 1.0285 std
nBase has 0 % of nan and 0.6192 std
SpAbs_A has 0 % of nan and 10.9784 std
SpMax_A has 0 % of nan and 0.2227 std
SpDiam_A has 0 % of nan and 0.4504 std
SpAD_A has 0 % of nan and 10.9784 std
SpMAD_A has 0 % of nan and 0.0782 std
LogEE_A has 0 % of nan and 0.5038 std
VE1_A has 0 % of nan and 0.7074 std
VE2_A has 0 % of nan and 0.0762 std
VE3_A has 0 % of nan and 0.6789 std
VR1_A has 0 % of nan and 285.6152 std
VR2_A has 0 % of nan and 6.5190 std
VR3_A has 0 % of nan and 1.4140 std
nAromAtom has 0 % of nan and 6.1459 std
nAromBon

In [53]:
from pteproc_model import PreprocessingModel

In [54]:
# p_dorp means we drop columns with more than 5% of nan values
model = PreprocessingModel(p_drop=0.1, threshold=1e-3, normalizer="minmax")

In [55]:
transformed_train = model.fit_transform(train)

In [56]:
transformed_train.shape

(5437, 1063)

In [57]:
pd.options.display.max_columns = 100
transformed_train

,pld,lcd,a,b,c,alpha,beta,gamma,volume,spacegroupNumber,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,VE3_A,VR2_A,VR3_A,nAromAtom,nAromBond,nAtom,nHeavyAtom,nHetero,nH,nC,nN,nO,nS,nP,nF,nCl,nBr,nI,nX,ATS0dv,ATS1dv,ATS2dv,ATS3dv,ATS4dv,ATS5dv,ATS6dv,ATS7dv,ATS8dv,...,JGI2,JGI3,JGI4,JGI5,JGI6,JGI7,JGI8,JGI9,JGI10,JGT10,Diameter,Radius,TopoShapeIndex,PetitjeanIndex,Vabc,VAdjMat,MWC01,MWC02,MWC03,MWC04,MWC05,MWC06,MWC07,MWC08,MWC09,MWC10,TMWC10,SRW02,SRW04,SRW05,SRW06,SRW07,SRW08,SRW09,SRW10,TSRW10,MW,AMW,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,n_metals,Atomic_Number,Atomic_Weight,Atomic Radius,Mulliken EN,polarizability(A^3),electron affinity(kJ/mol)
boydwoo_str_m7_Zn_o8_o25_bcu_sym_88,0.279007,0.308983,0.506393,0.475522,0.303016,0.777722,0.755360,0.767371,0.133105,0.000000,0.245173,0.308587,0.000000,0.0,0.229814,0.742956,0.737989,0.229814,0.915096,0.679468,0.334097,0.126280,0.616220,0.256530,0.745375,0.076923,0.066667,0.243976,0.231132,0.135135,0.235294,0.213542,0.208333,0.035714,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.199409,0.238074,0.224981,0.169244,0.106275,0.076342,0.078688,0.067367,0.041997,...,0.451008,0.362882,0.318674,0.263049,0.358981,0.418076,0.409178,0.230130,0.139280,0.532499,1.350000e-07,6.500000e-08,0.933036,0.964103,0.222777,0.709426,0.240000,0.745114,0.788920,0.823197,0.850862,0.851153,0.847338,0.844063,0.841777,0.840000,0.401466,0.685854,0.779823,0.692811,0.853887,0.786878,0.859393,0.826786,0.850572,0.441870,0.143914,0.095860,0.200508,0.256515,0.275232,0.156900,0.211466,0.000000,0.303371,0.252930,0.133333,0.841924,0.018450,0.039216
gmof_Al2O6-ADC_B-irmof20_B_No336,0.296299,0.255808,0.105333,0.393713,0.237160,0.578224,0.526884,0.471654,0.039509,0.000000,0.132209,0.197281,0.142857,0.0,0.111125,0.629338,0.567648,0.111125,0.700456,0.528758,0.265887,0.250702,0.497023,0.021795,0.505025,0.102564,0.100000,0.084337,0.122642,0.162162,0.014706,0.083333,0.000000,0.142857,0.2,0.0,0.071429,0.0,0.000000,0.0,0.050000,0.172239,0.131912,0.147806,0.095275,0.098545,0.032309,0.039863,0.017537,0.025258,...,0.441176,0.389145,0.389161,0.299587,0.563810,0.692308,0.000000,0.000000,0.000000,0.644429,7.000000e-08,3.000000e-08,1.000000,1.000000,0.102486,0.573289,0.120000,0.618790,0.669402,0.711290,0.742709,0.748986,0.749493,0.749678,0.749799,0.749973,0.288916,0.540703,0.649607,0.692811,0.721952,0.821591,0.730159,0.878334,0.725047,0.344300,0.097360,0.366687,0.111675,0.127036,0.126162,0.142509,0.100251,0.000000,0.112360,0.086723,0.000000,0.422680,0.053506,0.439216
MOF-5_cooh_2_1005_1,0.170161,0.401192,0.492708,0.468986,0.276424,0.581951,0.954855,0.867782,0.109402,0.000000,0.140822,0.204339,0.142857,0.0,0.126576,0.531432,0.531432,0.126576,0.683250,0.541732,0.280179,0.218368,0.523772,0.026223,0.539748,0.076923,0.066667,0.168675,0.141509,0.108108,0.191176,0.125000,0.041667,0.142857,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.152628,0.121793,0.108545,0.067227,0.060764,0.049135,0.053655,0.047891,0.036916,...,0.352941,0.145991,0.143395,0.232955,0.314386,0.406593,0.275510,0.000000,0.000000,0.448437,7.000000e-08,3.000000e-08,1.000000,1.000000,0.149138,0.585825,0.128000,0.609282,0.642244,0.673181,0.694536,0.695986,0.692041,0.689779,0.687277,0.685926,0.284547,0.553971,0.635671,0.000000,0.691917,0.000000,0.691768,0.000000,0.681848,0.250592,0.093408,0.081332,0.096447,0.120521,0.106242,0.147009,0.132832,0.272727,0.303371,0.252930,0.133333,0.841924,0.018450,0.039216
gmof_Zn4O13-BDC_A-irmof20_A_No248,0.246820,0.434390,0.504209,0.470985,0.567572,0.465501,0.368998,0.119818,0.237635,0.000000,0.137459,0.207823,0.214286,0.0,0.115238,0.608620,0.582069,0.115238,0.635200,0.536522,0.278207,0.238400,0.512315,0.021073,0.512585,0.089744,0.083333,0.102410,0.132075,0.162162,0.044118,0.093750,0.000000,0.214286,0.1,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.183166,0.146639,0.163395,0.121419,0.104415,0.073735,0.043736,0.035075,0.012629,...,0.448179,0.326223,0.374393,0.314354,0.534074,0.173077,0.000000,0.000000,0.000000,0.612830,6.5000

In [58]:
for col in transformed_train.columns:
    print(f"{col} has {((5437 - len(train[col].dropna()))/5437)*100:.0f} % of nan, {transformed_train[col].std():.4f} std, number of unique: {(len(np.unique(transformed_train[col]))):.0f}")

pld has 0 % of nan, 0.1427 std, number of unique: 5407
lcd has 0 % of nan, 0.1569 std, number of unique: 5416
a has 0 % of nan, 0.1476 std, number of unique: 5437
b has 0 % of nan, 0.1523 std, number of unique: 5437
c has 0 % of nan, 0.1270 std, number of unique: 5435
alpha has 0 % of nan, 0.1524 std, number of unique: 5347
beta has 0 % of nan, 0.1785 std, number of unique: 5335
gamma has 0 % of nan, 0.1947 std, number of unique: 5361
volume has 0 % of nan, 0.0694 std, number of unique: 5415
spacegroupNumber has 0 % of nan, 0.1910 std, number of unique: 140
ABC has 0 % of nan, 0.0773 std, number of unique: 2809
ABCGG has 0 % of nan, 0.0908 std, number of unique: 3302
nAcid has 0 % of nan, 0.0735 std, number of unique: 33
nBase has 0 % of nan, 0.0422 std, number of unique: 27
SpAbs_A has 0 % of nan, 0.0750 std, number of unique: 3323
SpMax_A has 0 % of nan, 0.0934 std, number of unique: 3226
SpDiam_A has 0 % of nan, 0.0944 std, number of unique: 3227
SpAD_A has 0 % of nan, 0.0750 std, n

In [59]:
transformed_test = model.transform(test)

In [60]:
transformed_train.shape, transformed_test.shape

((5437, 1063), (1360, 1063))

In [61]:
transformed_train.to_csv("../qmof_datasets/train.csv")
transformed_test.to_csv("../qmof_datasets/test.csv")
joblib.dump(model, "../qmof_datasets/scaler.pkl")

['../qmof_datasets/scaler.pkl']